<center><img src=img/MScAI_brand.png width=70%></center>

# Measuring Object Complexity

We have discussed computational complexity, and argued that both time complexity and space complexity are really the same concept, applied to two different quantities - number of instructions executed during runtime, and amount of space used during runtime.

But complexity is a term which has a lot of meanings in maths, physics, and AI. An important meaning in AI is: how complex is an object? Notice this is not closely related to computational complexity.

Another way of saying the same thing is: **how much information** is there in the object.

In this notebook we'll briefly look at this idea.

### Focus on strings

* We are not necessarily talking about objects in the OOP sense, but rather data structures in general. 

* Usually, we just focus on strings, because every computational object can be converted to a string somehow. 

* If we were talking about physical objects (eg, complexity of Notre Dame cathedral versus complexity of a barn) or even about other types of information (eg complexity of a piece of music by Beethoven, versus a nursery rhyme), we could still do it by somehow encoding a complete description of the object as a string. So, we focus on strings from now on!

### Examples

Consider the string:

In [9]:
s1 = 'aaaaaaaaaaaaaaaaaaaaaaaa'

The pattern in this string is very obvious! Even though there are 24 characters, we might say there is not much complexity, not much information.

Now consider another string (drawn from the same alphabet):

In [10]:
s2 = 'abcdabcdabcdabcdabcdabcd'

This time, there is more complexity, but still a very obvious pattern.

Now consider a third string. It doesn't seem that there is any pattern:

In [11]:
s3 = 'abaabddbcbabdbabcccadacc'

We might say:

`s1` contains the least information (lowest complexity), `s2` is in the middle (medium complexity), and `s3` contains the most (highest complexity).

```
s1 = 'aaaaaaaaaaaaaaaaaaaaaaaa'
s2 = 'abcdabcdabcdabcdabcdabcd'
s3 = 'abaabddbcbabdbabcccadacc'
```

### Measuring complexity

How can we quantify what we have observed?

One method is to calculate the **number of distinct $n$-grams**. 

If there are just a few $n$-grams in the string, it proves there must be a lot of repetition. If there are many distinct ones, it proves there is less repetition.

In [18]:
def count_distinct_ngrams(s, n):
    ngrams = [s[i:i+n] for i in range(len(s) - n + 1)]
    # print(len(ngrams), ngrams)
    return len(set(ngrams)) # count distinct ones


In [20]:
for s in (s1, s2, s3):
    print(f'{s}: {count_distinct_ngrams(s, 4)}')

aaaaaaaaaaaaaaaaaaaaaaaa: 1
abcdabcdabcdabcdabcdabcd: 4
abaabddbcbabdbabcccadacc: 21


This seems to be giving the right idea: it tells us the first string is extremely simple and the last is more complex.

### Kolmogorov complexity

Our $n$-gram method works ok for a lot of cases, but it's not the only method.

Another method is based on the observation: if we can **compress** the string, it proves we have identified there is pattern in it. More pattern implies less complexity (less information). 

For example, consider again: `s2 = 'abcdabcdabcdabcdabcdabcd'`. We can write a shorter string which is equivalent:

In [ ]:
'abcd' * 6 

'abcdabcdabcdabcdabcdabcd'

This can be much very powerful. For some strings our $n$-grams method will give a high value, even though the human eye can see there is a pattern. For example:

In [23]:
s4 = '112123123412345123456123456712345678'
count_distinct_ngrams(s4, 4)

23

Probably we could compress `s4` by **writing a short program** which outputs `s4`. (**Exercise**: try it.)

Based on this, we define the **Kolmogorov complexity** for a string $s$: **the length of the shortest program which would output $s$**.

Unfortunately, finding the **shortest program which outputs a given string** is **uncomputable**: no algorithm can guarantee to do it!

### Some other practical methods

So, in practice we use simple methods, eg:

* Our method, counting distinct $n$-grams
* Entropy (are all symbols equally common in the string?)
* Conditional entropy (are all $n$-grams equally common in the string?)
* LZW-compression (related to the algorithms used in zip compression, see eg https://rosettacode.org/wiki/LZW_compression#Python)
* Assembly (how many "joins" do we need to do to assemble the string from single letters, allowing for reuse of substrings?).
* More generally, the field of **Algorithmic Information Theory** is about these kinds of concepts.

If you would like to see some of these, read on. These are not examinable.

In [24]:
from math import log2
from itertools import product
from sksequitur import parse # pip install scikit-sequitur

In [37]:
def condent(s, n=2, alphabet=None, k=1):
    '''
    Conditional entropy
    Using n-grams as the "predictors"
    Using optional smoothing
    Using optional specification of a larger alphabet
    '''

    # Initialise alphabet if not specified and check
    if alphabet == None:
        alphabet = sorted(''.join(set(s)))
    assert all((x in alphabet for x in s))

    # Initialize smoothed counts for single symbols and n-grams
    # ie create a value of k for every possible single symbol
    # and every possible (n-1)-gram and every possible n-gram
    single_freqs = {symbol: k for symbol in alphabet}
    ngram_freqs = {''.join(gram): k for gram in product(alphabet, repeat=n)}
    n_1_gram_freqs = {''.join(gram): k for gram in product(alphabet, repeat=n-1)}
    
    # Count n-gram frequencies, updating smoothed counts
    for i in range(len(s) - n + 1):
        ngram = s[i:i + n]
        if ngram in ngram_freqs:
            ngram_freqs[ngram] += 1

    # Count (n-1)-gram frequencies, updating smoothed counts
    for i in range(len(s) - n + 2):
        n_1_gram = s[i:i + n - 1]
        if n_1_gram in n_1_gram_freqs:
            n_1_gram_freqs[n_1_gram] += 1

    # Calculate conditional probabilities based on smoothed counts
    cond_prob = {ngram: count / n_1_gram_freqs[ngram[:-1]] for ngram, count in ngram_freqs.items()}
    
    # Calculate conditional entropy
    entropy = -sum(prob * log2(prob) for prob in cond_prob.values())
    
    return entropy

for s in (s1, s2, s3):
    print(f'{s}: {condent(s, n=4, alphabet="abcd"):.3f}')


aaaaaaaaaaaaaaaaaaaaaaaa: 0.651
abcdabcdabcdabcdabcdabcd: 5.182
abaabddbcbabdbabcccadacc: 31.173


In [33]:
def assembly_index(s):
    '''
    Assembly index as defined by Cronin and colleagues
    g is a "grammar"

    eg for s = 'abcabcabc', we get a grammar of two rewrite rules:
    0 -> 1 1 1
    1 -> a b c 

    Each rule of length k require (k-1) joins 
    (eg '1 1 1' requires 2 joins)
    hence our formula summing len(g[k]) - 1
    '''
    g = parse(s) 
    # each rule in the grammar
    return sum(len(g[k]) - 1 for k in g)


for s in (s1, s2, s3):
    print(f'{s}: {assembly_index(s)}')


aaaaaaaaaaaaaaaaaaaaaaaa: 5
abcdabcdabcdabcdabcdabcd: 6
abaabddbcbabdbabcccadacc: 18
